In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate


# 3.3 - leapfrog and the Driven Harmonic Oscillator


Solving the harmonic oscillator by defining it as an ODE and solving the initial value problem:

parameters m=k=1

initial condition v(-0.5) = 0, x(0)=1

First we solve the system with different methods and measure the amplitude over long timescales (this cell runs for 3 minutes)

In [ ]:
m=1
k=1
t_max=1000000
t_steps = 10*t_max

def f_t(t):
    return 0


def f(t, y):
    x, v = y
    dx = v
    dv = (f_t(t) - k*x)/m
    return np.array([dx, dv])

y0 = np.array([1,0])
ts, dt = np.linspace(0,t_max,t_steps, retstep=True)

plt.figure(figsize=[6,3])
for method in ['RK23', 'RK45', 'DOP853']:

    sol = scipy.integrate.solve_ivp(f, [0,t_max], y0=y0, t_eval=ts, method=method)
    y_max = np.max(sol.y[0].reshape(-1,100), axis=1)
    plt.plot(ts[::100], y_max, label=method)

x = np.zeros_like(ts)
v = np.zeros_like(ts)
x[0]=1
for i, t in enumerate(ts[:-1]):
    v[i+1] = v[i] + dt*(f_t(t) - k*x[i])
    x[i+1] = x[i] + dt*v[i+1]
       
x_max = np.max(x.reshape(-1,100), axis=1)
plt.plot(ts[::100], x_max, label='leapfrog')
plt.grid()
plt.ylabel(r'$x_{max}$', fontsize=16)
plt.xlabel('t', fontsize=16)
plt.xscale('log')
plt.legend(title='method')

plt.savefig('../results/solver_comare.png', dpi=600)


Now we add a sinusoidal driving force with frequency $w_{driv}$ to the oscillator and solve it with the leapfrog method 

In [ ]:
fig1, ax1 = plt.subplots(1,3, figsize=[6,2])
fig2, ax2 = plt.subplots(3,1, figsize=[6,6], sharex=True)
for n, wdriv in enumerate([2., 1.5, 1.1]):
    m=1
    k=1
    t_max=100
    t_steps = 100*t_max

    def f_t(t):
        return np.sin(wdriv*t)


    def f(t, y):
        x, v = y
        dx = v
        dv = (f_t(t) - k*x)/m
        return np.array([dx, dv])

    y0 = np.array([0,5])
    ts, dt = np.linspace(0,t_max,t_steps, retstep=True)


    x = np.zeros_like(ts)
    v = np.zeros_like(ts)
    x[0] =1
    for i, t in enumerate(ts[:-1]):
        v[i+1] = v[i] + dt*(f_t(t) - k*x[i])
        x[i+1] = x[i] + dt*v[i+1]
    ax2[n].plot(ts, x, label='x')
    ax2[n].plot(ts - 0.5*dt, v, label='v')
    ax2[n].legend(loc=2)

    ax1[n].set_xlabel('x')
    ax1[n].scatter(x, v, s=1, marker='.')
    
    ax1[n].set_title(r'$\omega_D = {}$'.format(wdriv))
    ax2[n].set_title(r'$\omega_D = {}$'.format(wdriv))

ax2[-1].set_xlabel('t')
ax1[0].set_ylabel('v')
fig1.tight_layout()
fig2.tight_layout()

fig1.savefig('../results/phase_diags.png', dpi=600)
fig2.savefig('../results/xv.png', dpi=600)

    